In [ ]:
%matplotlib inline
import sys
sys.path.append("..")
from importlib import reload
import numpy as np
from matplotlib import pyplot as plt
import igraph

import pythd

In [ ]:
# Build the dataset
reload(pythd)
reload(pythd.datagen)
datagen = (pythd.datagen.DatasetGenerator()
                .random_disk(center=[-3.0, 0.0], min_radius=0.8, max_radius=1.4, num_points=300)
                .gaussian(center=[3.0, 0.0], sd=[0.5, 0.5], num_points=1000)
                .gaussian(center=[0.0, -3.0], sd=[0.5, 0.5], num_points=1000)
                .arc(center=[0.0, 3.0], radius=1.5, num_points=1000, noise=0.15)
                .line(start=[-2.1, 0], end=[2.0, 0.0], num_points=1000, noise=0.2))
dataset = datagen.get()
left_rids = [i for i, p in enumerate(dataset) if p[0] < -3.0]
right_rids = [i for i, p in enumerate(dataset) if p[0] > 3.0]
middle_rids = [i for i, p in enumerate(dataset) if ((p[1] < 2.0) and (p[1] > -2.0))]
middle_line = [i for i, p in enumerate(dataset) if ((i in middle_rids) and (p[0] > -2.0) and (p[0] < 2.0))]
plt.plot(dataset[:, 0], dataset[:, 1], ".")
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()
plt.show()

In [ ]:
# Run THD
reload(pythd)
reload(pythd.thd)
reload(pythd.complex)
reload(pythd.cover)
reload(pythd.mapper)
reload(pythd.clustering)

filt = pythd.filter.IdentityFilter()
f_x = filt(dataset)
clustering = pythd.clustering.HierarchicalClustering()
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 10, 0.3)
thd = pythd.thd.THD(dataset, filt, cover, group_threshold=180)
thd.run()
root = thd.get_results()

In [ ]:
import igraph
root.color_by_rids(middle_line, normalize=True)
g = root.as_igraph_graph()
vs = {
    "margin": 40,
    "bbox": (350, 350),
    "vertex_label": g.vs["name"],
    "vertex_label_size": 10,
    "vertex_size": 20,
    "vertex_label_dist": 1.5,
    "vertex_label_angle": 0,
    "layout": g.layout_reingold_tilford(root=[0])
}
igraph.plot(g, **vs)

In [ ]:
mid = root.get_group_by_name("1.2.0")
g = mid.network.get_igraph_network()
layout = g.layout_kamada_kawai(maxiter=3000)
pythd.plotting.draw_topological_network(mid.network, layout)

In [ ]:
reload(pythd.plotting)
pythd.plotting.plot_2d_point_cloud(mid.get_data().values, xlim=xlim, ylim=ylim)

In [ ]:
pythd.plotting.plot_2d_point_cloud(root.get_group_by_name("1.1.0").get_data().values, xlim=xlim, ylim=ylim)

In [ ]:
pythd.plotting.plot_2d_point_cloud(root.get_group_by_name("1.0.0").get_data().values, xlim=xlim, ylim=ylim)

In [ ]:
mid = root.get_group_by_name("4.2.2")
g = mid.network.get_igraph_network()
layout = g.layout_kamada_kawai(maxiter=3000)
pythd.plotting.draw_topological_network(mid.network, layout)